In [4]:
import json
from tqdm import tqdm_notebook as tqdm
import os
raw_data_path = './raw_data/cornell'
prepared_data_path = './prepared_data/cornell'

In [5]:
def load_lines(file_path):
    lines_dict = {}
    with open(file_path, 'r', encoding='iso-8859-1') as file:  # TODO: Solve Iso encoding pb !
        for line in file:
            splitted = line.split(' +++$+++ ')
            
            line_id = splitted[0]
            character_id = splitted[1]
            text = splitted[4]
            
            lines_dict[line_id] = text

    return lines_dict

def load_conversations(file_path, lines):
    
    conversations = []

    with open(file_path, 'r', encoding='iso-8859-1') as file:
        for line in file:
            values = line.split(' +++$+++ ')
            line_ids = json.loads(values[3].replace("'", '"'))
            
            conversation = []
            for line_id in line_ids:
                conversation.append(lines[line_id])
            conversations.append(conversation)
        
    return conversations

def merge_q_and_a(conversations):
    
    merged = {}
    for conversation in conversations:
        # Iterate over all the lines of the conversation
        for i in range(len(conversation) - 1):  # We ignore the last line (no answer for it)
            question = conversation[i].strip()
            answer = conversation[i+1].strip()

            if question and answer:  # Filter wrong samples (if one of the list is empty)
                merged[question] = answer

    return merged



In [7]:
lines = load_lines(os.path.join(raw_data_path, 'movie_lines.txt'))
conversations = load_conversations(os.path.join(raw_data_path, 'movie_conversations.txt'), lines)
merged = merge_q_and_a(conversations)

In [8]:
if not os.path.exists(prepared_data_path):
    os.makedirs(prepared_data_path)
with open(os.path.join(prepared_data_path, 'questions_cornell.txt'), 'w') as q_file, open(os.path.join(prepared_data_path, 'answers_cornell.txt'), 'w') as a_file:
    for question, answer in tqdm(merged.items()):
        q_file.write(question + '\n')
        a_file.write(answer + '\n')